In [ ]:
#!pip install tensorflow-hub-0.12.0
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

#!pip install tensorflow-privacy
import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.membership_inference_attack as mia
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackType

In [ ]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [5]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))
print(train_examples[:10])
print(train_labels[:10])

Training entries: 25000, test entries: 25000
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because

In [6]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

2022-07-15 21:06:31.475164: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.


<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [9]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [10]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=10,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/10


2022-07-15 21:07:06.066005: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.
2022-07-15 21:07:06.137936: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.
2022-07-15 21:07:07.085425: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.
2022-07-15 21:07:07.085553: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.


30/30 [==============================] - 29s 891ms/step - loss: 0.6574 - accuracy: 0.6377 - val_loss: 0.6090 - val_accuracy: 0.7198
Epoch 2/10
30/30 [==============================] - 26s 869ms/step - loss: 0.5365 - accuracy: 0.7784 - val_loss: 0.4948 - val_accuracy: 0.7869
Epoch 3/10
30/30 [==============================] - 26s 861ms/step - loss: 0.3927 - accuracy: 0.8583 - val_loss: 0.3893 - val_accuracy: 0.8410
Epoch 4/10
30/30 [==============================] - 25s 842ms/step - loss: 0.2767 - accuracy: 0.9050 - val_loss: 0.3353 - val_accuracy: 0.8608
Epoch 5/10
30/30 [==============================] - 26s 872ms/step - loss: 0.2004 - accuracy: 0.9370 - val_loss: 0.3123 - val_accuracy: 0.8695
Epoch 6/10
30/30 [==============================] - 26s 871ms/step - loss: 0.1448 - accuracy: 0.9596 - val_loss: 0.3039 - val_accuracy: 0.8735
Epoch 7/10
30/30 [==============================] - 25s 849ms/step - loss: 0.1038 - accuracy: 0.9747 - val_loss: 0.3046 - val_accuracy: 0.8744
Epoch 8/10

In [11]:
print('Predict on train...')
logits_train = model.predict(partial_x_train)
print('Predict on test...')
logits_test = model.predict(x_val)

print('Apply softmax to get probabilities from logits...')
prob_train = tf.nn.softmax(logits_train, axis=-1)
prob_test = tf.nn.softmax(logits_test)

print('Compute losses...')
cce = tf.keras.backend.categorical_crossentropy
constant = tf.keras.backend.constant


Predict on train...
Predict on test...
Apply softmax to get probabilities from logits...
Compute losses...


In [12]:
y_train_onehot = partial_y_train.reshape(partial_y_train.shape[0], 1)
y_test_onehot = y_val.reshape(y_val.shape[0], 1)

loss_train = cce(constant(y_train_onehot), constant(prob_train), from_logits=False).numpy()
loss_test = cce(constant(y_test_onehot), constant(prob_test), from_logits=False).numpy()

In [13]:
attack_input = AttackInputData(
  logits_train = logits_train,
  logits_test = logits_test,
  loss_train = loss_train,
  loss_test = loss_test,
  labels_train = partial_y_train,
  labels_test = y_val
)

slicing_spec = SlicingSpec(
    entire_dataset = True,
    by_class = True,
    by_percentiles = False,
    by_classification_correctness = True)

attack_types = [    
    AttackType.K_NEAREST_NEIGHBORS,    
    AttackType.LOGISTIC_REGRESSION
]

In [14]:
attacks_result = mia.run_attacks(attack_input=attack_input,
                                 slicing_spec=slicing_spec,
                                 attack_types=attack_types)

In [15]:
print(attacks_result.summary(by_slices=True))

Best-performing attacks over all slices
  LOGISTIC_REGRESSION (with 4998 training and 4998 test examples) achieved an AUC of 0.64 on slice CLASS=0
  LOGISTIC_REGRESSION (with 4998 training and 4998 test examples) achieved an advantage of 0.27 on slice CLASS=0

Best-performing attacks over slice: "Entire dataset"
  K_NEAREST_NEIGHBORS (with 10000 training and 10000 test examples) achieved an AUC of 0.59
  K_NEAREST_NEIGHBORS (with 10000 training and 10000 test examples) achieved an advantage of 0.14

Best-performing attacks over slice: "CLASS=0"
  LOGISTIC_REGRESSION (with 4998 training and 4998 test examples) achieved an AUC of 0.64
  LOGISTIC_REGRESSION (with 4998 training and 4998 test examples) achieved an advantage of 0.27

Best-performing attacks over slice: "CLASS=1"
  LOGISTIC_REGRESSION (with 5002 training and 5002 test examples) achieved an AUC of 0.63
  LOGISTIC_REGRESSION (with 5002 training and 5002 test examples) achieved an advantage of 0.24

Best-performing attacks over 

In [16]:
max_auc_attacker = attacks_result.get_result_with_max_auc()
max_advantage_attacker = attacks_result.get_result_with_max_attacker_advantage()

In [17]:
print("Attack type with max AUC: %s, AUC of %.2f, Attacker advantage of %.2f" %
      (max_auc_attacker.attack_type,
       max_auc_attacker.roc_curve.get_auc(),
       max_auc_attacker.roc_curve.get_attacker_advantage()))


Attack type with max AUC: LOGISTIC_REGRESSION, AUC of 0.64, Attacker advantage of 0.27
